### Домашнее задание

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, precision_score, \
classification_report, precision_recall_curve, confusion_matrix
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
df['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

In [4]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = \
train_test_split(df, df['Exited'], random_state=0)

In [5]:
#соберем простой pipeline
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Преобразователь для выбора одного столбца из data frame для выполнения 
    дополнительных преобразований
    Использовать в числовых столбцах данных
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

### Логистическая регрессия

In [7]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
        ('selector', FeatureSelector(column=cat_col)),
        ('ohe', OHEEncoder(key=cat_col))
    ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)), 
        ('standard', StandardScaler())
    ])
    final_transformers.append((cont_col, cont_transformer))

In [8]:
feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

In [9]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(class_weight={0: 1,1: 4},
                                      random_state = 42))
])

In [10]:
#обучим пайплайн
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [11]:
#прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]

In [12]:
def get_metrics(y_test, preds):
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    # индекс наибольшего значения fscore
    ix = np.argmax(fscore)
    print('Best Threshold = %.3f, \nF-Score = %.3f, \nPrecision = %.3f, \
          \nRecall = %.3f, \nRoc-AUC = %.3f' % (thresholds[ix], fscore[ix], 
            precision[ix], recall[ix], roc_auc_score(y_test, preds)))
    return {'Best Threshold': thresholds[ix], 
            'F-Score': fscore[ix], 
            'Precision': precision[ix], 
            'Recall': recall[ix], 
            'Roc-AUC': roc_auc_score(y_test, preds)}

In [13]:
metrics_LR = get_metrics(y_test, preds)
thresholds = metrics_LR['Best Threshold']

Best Threshold = 0.570, 
F-Score = 0.515, 
Precision = 0.427,           
Recall = 0.648, 
Roc-AUC = 0.775


In [14]:
# матрица ошибок
cnf_matrix_LR = confusion_matrix(y_test, preds>thresholds)
cnf_matrix_LR

array([[1549,  442],
       [ 180,  329]], dtype=int64)

### Бустинг

In [15]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

In [16]:
feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

In [17]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(max_depth=3, min_samples_leaf=3, 
                                              n_estimators=50, random_state=42))
])

In [18]:
#обучим наш пайплайн
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [19]:
#прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]

In [20]:
metrics_GBC = get_metrics(y_test, preds)
thresholds = metrics_GBC['Best Threshold']

Best Threshold = 0.319, 
F-Score = 0.643, 
Precision = 0.630,           
Recall = 0.656, 
Roc-AUC = 0.872


In [21]:
# матрица ошибок
cnf_matrix_GBC = confusion_matrix(y_test, preds>thresholds)
cnf_matrix_GBC

array([[1796,  195],
       [ 176,  333]], dtype=int64)

###  Вывод

In [22]:
pd.DataFrame([metrics_LR, metrics_GBC], 
             index=['Logistic Regression', 'Gradient Boosting Classifier'])

,Best Threshold,F-Score,Precision,Recall,Roc-AUC
Logistic Regression,0.570240,0.515222,0.427461,0.648330,0.774718
Gradient Boosting Classifier,0.318552,0.642926,0.630189,0.656189,0.871549


- Лучшая модель: GradientBoostingClassifier  
- Наиболее подходящая метрика: Precision

### Оценка экономической эффективности

In [23]:
# матрица лучшей модели
cnf_matrix_GBC

array([[1796,  195],
       [ 176,  333]], dtype=int64)

In [24]:
def profit_score(cnf_matrix, m, n):
    TP, FP, FN, TN = cnf_matrix_GBC.reshape(1, 4)[0]
    print(pd.DataFrame([{'TP': TP, 'FP': FP, 'FN': FN,'TN': TN}]), '\n')
    
    # Прибыль с удержанием:
    P1 = (TP * m) + (FP * m) - (FN * n) + (TN * n)
    print(f'Прибыль с удержанием: {P1}')
    # Прибыль без удержания:
    P2 = - (TP * n) - (FN * n) + (FP * n) + (TN * n)
    print(f'Прибыль без удержанием: {P2}')
    
    print('\nМодель с удержанием является экономически {}целесообразной.'
          .format('' if P1 > P2 else 'не'))

In [25]:
profit_score(cnf_matrix_GBC, 1, 2)

     TP   FP   FN   TN
0  1796  195  176  333 

Прибыль с удержанием: 2305
Прибыль без удержанием: -2888

Модель с удержанием является экономически целесообразной.


### Подбор гиперпараметров лучшей модели

In [26]:
params = {
    'classifier__max_depth': [3, 5, 7],
    'classifier__min_samples_leaf': [3, 5, 7],
    'classifier__n_estimators': [50, 100, 150]    
}

In [27]:
%%time
grid = GridSearchCV(pipeline, param_grid=params)
search = grid.fit(X_train, y_train)
search.best_params_

Wall time: 3min 51s


{'classifier__max_depth': 5,
 'classifier__min_samples_leaf': 3,
 'classifier__n_estimators': 100}

In [28]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(
        **{i[len('classifier__'):]: j for i, j in search.best_params_.items()},
        random_state=42))
])

In [29]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict_proba(X_test)[:, 1]

In [30]:
metrics_GBC = get_metrics(y_test, preds)
thresholds = metrics_GBC['Best Threshold']

Best Threshold = 0.404, 
F-Score = 0.632, 
Precision = 0.663,           
Recall = 0.603, 
Roc-AUC = 0.873


In [31]:
# матрица ошибок
cnf_matrix_GBC = confusion_matrix(y_test, preds>thresholds)
cnf_matrix_GBC

array([[1835,  156],
       [ 203,  306]], dtype=int64)

### Оценка экономической эффективности

In [32]:
profit_score(cnf_matrix_GBC, 1, 2)

     TP   FP   FN   TN
0  1835  156  203  306 

Прибыль с удержанием: 2197
Прибыль без удержанием: -3152

Модель с удержанием является экономически целесообразной.
